## Machine Learning - Final Project - Winter 2023 - Group 16 

#### RIOT Games Data Collection for League of Legends

We will collect the game data for our project using the RIOT Games API and Python. 

* We will be using data from the highest quality of games, which generally comes from games played by the highest ranked players
* We will be ommiting duplicate games
* The game data will be collected from a single patch (version of the game) 
* The game data will be collected from the North America Region

In [1]:
import os
from dotenv import load_dotenv
from riotwatcher import LolWatcher, ApiError
import numpy as np
import pandas as pd
import sqlite3

In [2]:
API_KEY = os.getenv("API_KEY")

In [3]:
watcher = LolWatcher(API_KEY)
region = 'NA1'

# Sample In-game-name we can use for single data collection
IGN = "ARKE9"

In [4]:
# Each account in the game has a unique id, we use this unique id to access an accounts match history and statistics

def get_summoner_uid(name):
    try:
        summoner = watcher.summoner.by_name(region, name)
    except ApiError:
        return "No summoner found"
    return summoner["puuid"]

player_uid = get_summoner_uid(IGN)

In [5]:
# With the unique id, we can access the player's data
# There are different ranked modes, we access solo/duo matched by specifying the queue id

match_list = watcher.match.matchlist_by_puuid('americas', player_uid, count=1, queue=420, type="ranked")

# We are returned with 20 match ids. We can be specific with how many matches we want as long as it is less than 20 total
print(match_list)


['NA1_4589883519']


In [39]:
game_timeline = watcher.match.timeline_by_match('americas', match_list[0])
game_details = watcher.match.by_id('americas', match_list[0])

In [40]:
# Provides end of game match details
game_details

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4588387470',
  'participants': ['iZe2ZQ0W1YYx6L42ubK91zNYtfpNl9MekD0QwiE-BFAXtnJKQEL2Nfpew1liG8CIwtcv1zC4XNB3lA',
   'Uqx1O5Vf4TkcTzFJjGFubGQ-7Mt_nbWY4ncYLT2X4XWyPuCPZL7gVxu7pe0OINk2-Rrf8f8cIj9vdQ',
   'bDLheomX4HWAxul1cBgnK5xJPInQ0_lN2dA0myoOeYFfZgKkW5BrZiTrqrQbKoT8zVZggyqRr-phFA',
   'UUbB7y4Ch2Pg1oEgIRAC3EqiLZWekIXP5J446R39wTg4etfgfxw6TsY9Tg88wIDCd5qDTocwxwQZNA',
   '_CtQfP8AJSQPXLQm9TTdopM5t1o2dpXWRPs7cSo0waKb_i6K0dVC5X1EgShcgTR0AsyEDRWkCHSHVQ',
   'VcNMmc0mPhHPKo-WUmrReD5jfp_4F6hpbWLlCKpcILFMujLhf-uDurf193-HvbCb9M2IEnts0VcXXA',
   'NgojHC8PCzj7oRrux_IxqxFG-QrCCViVBIyYsweiTXXaluaEzCBKDp8M8640XC6mIYcF4PiRnYftLg',
   'zs2Rqn8HpCd5DpYCZAN_1gF7OHsZQo2I2Fp8DaV52EXVQbbUs-rsBQgpMk_jzVPE9lKD_2RdDJ1GVQ',
   'jNVqgwi8xetAFl9pgc9OA_feXM2ew176-cZ64XqiqKVl8An0Fc863JfKptkgUlYKlw7itQn_eO5NTg',
   'nqdiGRCJ6w9qzvOaNTJOyvXbh4yo4emLJi2bUM7rWVFapasG4Cs8Xf3OZLdbz7pRd6ocdsGH3dseIw']},
 'info': {'gameCreation': 1677471074260,
  'gameDuration': 2074,
  

In [41]:
# Provides timeline
game_timeline

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4588387470',
  'participants': ['iZe2ZQ0W1YYx6L42ubK91zNYtfpNl9MekD0QwiE-BFAXtnJKQEL2Nfpew1liG8CIwtcv1zC4XNB3lA',
   'Uqx1O5Vf4TkcTzFJjGFubGQ-7Mt_nbWY4ncYLT2X4XWyPuCPZL7gVxu7pe0OINk2-Rrf8f8cIj9vdQ',
   'bDLheomX4HWAxul1cBgnK5xJPInQ0_lN2dA0myoOeYFfZgKkW5BrZiTrqrQbKoT8zVZggyqRr-phFA',
   'UUbB7y4Ch2Pg1oEgIRAC3EqiLZWekIXP5J446R39wTg4etfgfxw6TsY9Tg88wIDCd5qDTocwxwQZNA',
   '_CtQfP8AJSQPXLQm9TTdopM5t1o2dpXWRPs7cSo0waKb_i6K0dVC5X1EgShcgTR0AsyEDRWkCHSHVQ',
   'VcNMmc0mPhHPKo-WUmrReD5jfp_4F6hpbWLlCKpcILFMujLhf-uDurf193-HvbCb9M2IEnts0VcXXA',
   'NgojHC8PCzj7oRrux_IxqxFG-QrCCViVBIyYsweiTXXaluaEzCBKDp8M8640XC6mIYcF4PiRnYftLg',
   'zs2Rqn8HpCd5DpYCZAN_1gF7OHsZQo2I2Fp8DaV52EXVQbbUs-rsBQgpMk_jzVPE9lKD_2RdDJ1GVQ',
   'jNVqgwi8xetAFl9pgc9OA_feXM2ew176-cZ64XqiqKVl8An0Fc863JfKptkgUlYKlw7itQn_eO5NTg',
   'nqdiGRCJ6w9qzvOaNTJOyvXbh4yo4emLJi2bUM7rWVFapasG4Cs8Xf3OZLdbz7pRd6ocdsGH3dseIw']},
 'info': {'frameInterval': 60000,
  'frames': [{'events': [{'realTi

In [42]:
# Frames represent each minute of the game. we have n+1 frames, we n is the match length in minutes rounded up to the nearest minute
frames = game_timeline['info']['frames']
len(frames)

36

In [43]:
if game_details["info"]["participants"][0]["win"] == True:
    winner = "Team 1"
else:
    winner = "Team 2"
winner

'Team 1'

In [44]:


gold = {}
for k in range(len(frames)):
    gold['{}'.format(k)] = {}
for value in gold.values():
    for l in range(1, 11):
        value['{}'.format(l)] = 0

for m in range(len(frames)):
    for n in range(1, 11):
        gold['{}'.format(m)]['{}'.format(n)] = game_timeline['info']['frames'][m]['participantFrames']['{}'.format(n)]['totalGold']

blue_gold = []
red_gold = []

for key in gold.keys():
    team1 = 0
    team2 = 0
    for o in range(1, 6):
        team1 += gold[key]['{}'.format(o)]
        team2 += gold[key]['{}'.format(o + 5)]
    blue_gold.append(team1)
    red_gold.append(team2)

gold, blue_gold, red_gold

({'0': {'1': 500,
   '2': 500,
   '3': 500,
   '4': 500,
   '5': 500,
   '6': 500,
   '7': 500,
   '8': 500,
   '9': 500,
   '10': 500},
  '1': {'1': 500,
   '2': 500,
   '3': 500,
   '4': 500,
   '5': 500,
   '6': 533,
   '7': 500,
   '8': 914,
   '9': 603,
   '10': 533},
  '2': {'1': 600,
   '2': 616,
   '3': 574,
   '4': 584,
   '5': 523,
   '6': 617,
   '7': 601,
   '8': 953,
   '9': 687,
   '10': 556},
  '3': {'1': 1050,
   '2': 1141,
   '3': 1082,
   '4': 902,
   '5': 718,
   '6': 816,
   '7': 951,
   '8': 1514,
   '9': 1354,
   '10': 769},
  '4': {'1': 1436,
   '2': 1364,
   '3': 1370,
   '4': 1239,
   '5': 913,
   '6': 1022,
   '7': 1178,
   '8': 1718,
   '9': 1669,
   '10': 1005},
  '5': {'1': 1752,
   '2': 1561,
   '3': 2090,
   '4': 1473,
   '5': 1509,
   '6': 1312,
   '7': 1300,
   '8': 2067,
   '9': 2029,
   '10': 1440},
  '6': {'1': 2185,
   '2': 2157,
   '3': 2377,
   '4': 1905,
   '5': 1785,
   '6': 1536,
   '7': 1709,
   '8': 2436,
   '9': 2254,
   '10': 1595},
  '7': 

In [46]:
blue_dragons = 0
blue_heralds = 0
red_dragons = 0
red_heralds = 0
blue_barons = 0
red_barons = 0

for frame in frames:
    for event in frame["events"]:
        if "monsterType" in event.keys():
            if event["monsterType"] == "DRAGON":
                if event["killerTeamId"] == 100:
                    blue_dragons += 1
                else:
                    red_dragons += 1
            if event["monsterType"] == "RIFTHERALD":
                if event["killerTeamId"] == 100:
                    blue_heralds += 1
                else:
                    red_heralds += 1
            if event["monsterType"] == "BARON_NASHOR":
                if event["killerTeamId"] == 100:
                    blue_barons += 1
                else:
                    red_barons += 1
blue_dragons, red_dragons

(2, 3)

In [47]:
blue_k = 0 # blue side (bottom)
red_k = 0 # red side (top)

for frame in frames:
    for event in frame["events"]:
        if "victimId" in event.keys():
            if event["type"] == "CHAMPION_KILL":
                if event["killerId"] == 0:
                    continue
                if 1 <= event["killerId"] <= 5:
                    blue_k += 1
                else:
                    red_k += 1
                    
blue_obj = 0
red_obj = 0
for frame in frames:
    for event in frame["events"]:
        if "buildingType" in event.keys():
            if event["teamId"] == 100:
                red_obj += 1
            else:
                blue_obj += 1

blue_k, red_k

(39, 27)